<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/ArabicPIIRedaction/blob/main/fine-tuning_SLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [1]:
import pandas as pd

### Test data
- Original Competition Data

In [2]:
data_url = "https://raw.githubusercontent.com/MuhammadHelmyOmar/ArabicPIIRedaction/main/data/data_final.xlsx"

test = pd.read_excel(data_url)

test.head()

,source,target,dialect
0,الأستاذ أدولفوس ريغان زييمان، بصفتك المدير الت...,الأستاذ [MASK]، بصفتك المدير التنفيذي الرئيسي ...,EGYPT
1,مرحبًا يا هانا، ممكن لو سمحتي تتحققي من التداع...,مرحبًا يا [MASK]، ممكن لو سمحتي تتحققي من التد...,EGYPT
2,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,نطلب أيضًا مراجعة سياساتنا فيما يتعلق بالتغيير...,SAUDI
3,عزيزي ديفان، مطلوب تقديم عرض تقديمي على مستوى ...,عزيزي [MASK]، مطلوب تقديم عرض تقديمي على مستوى...,EGYPT
4,ممكن كمان نعمل جلسة عن كيفية إدارة التوتر والت...,The sentence doesn't contain any personal or s...,LEVANTINE


### Train data
- Created by Mosaab

In [3]:
from google.colab import drive
import random

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data_path = "/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/ar_dataset_translated_fixed.csv"

train = pd.read_csv(data_path)

In [6]:
train.drop(["Unnamed: 0",'lang'], axis=1, inplace=True)
print(len(train))
train.head()

209261


,source,tokens,tags,clean_source
0,مرحبًا <!FIRSTNAME_1!> Johan </!FIRSTNAME_1!> ...,"['مرحبا', 'Johan', 'Walker', '،', 'إشعار', 'ال...","['O', 'B-FIRSTNAME', 'B-LASTNAME', 'O', 'O', '...",مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...
1,نحن نقوم بمراجعة القضية التي رفعها <!FIRSTNAME...,"['نحن', 'نقوم', 'بمراجعة', 'القضية', 'التي', '...","['O', 'O', 'O', 'O', 'O', 'O', 'B-FIRSTNAME', ...",نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...
2,بسبب اتفاقية السرية، لا يمكن للمريضة <!SEX_1!>...,"['بسبب', 'اتفاقية', 'السرية', '،', 'لا', 'يمكن...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEX', '...",بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في <!DATE...,"['عزيزي', 'ولي', 'الأمر', '،', 'يلزم', 'الحصول...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...
4,تنظيم اجتماع حول <!AGE_1!> 7 سنوات </!AGE_1!> ...,"['تنظيم', 'اجتماع', 'حول', '7', 'سنوات', '(', ...","['O', 'O', 'O', 'B-AGE', 'I-AGE', 'O', 'B-GEND...",تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...


In [7]:
idx = random.randint(0, len(train))
idx

33257

In [8]:
print(train.tokens[idx])
print(train.tags[idx])
print(len(train.tokens[idx]))
print(len(train.tags[idx]))

['مرحبا', 'Reed', '،', 'لقد', 'واجهنا', 'مشكلة', 'في', 'معالجة', 'المدفوعات', 'الدولية', 'لمزود', 'تكنولوجيا', 'التعليم', 'الجديد', 'لدينا', '.', 'هل', 'يمكنك', 'التحقق', 'من', 'المشكلة', 'مع', 'BIC', 'LERQNZJBXXX', 'و', 'IBAN', 'NO2484520800325', '؟']
['O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-BIC', 'O', 'O', 'B-IBAN', 'O']
252
159


##### Cleaning and preprocessing

In [9]:
import re
import ast

In [10]:
tokens_list = ast.literal_eval(train.tokens[idx])
print(tokens_list)
print(type(tokens_list), len(tokens_list))

tags_list = ast.literal_eval(train.tags[idx])
print(tags_list)
print(type(tags_list), len(tags_list))

['مرحبا', 'Reed', '،', 'لقد', 'واجهنا', 'مشكلة', 'في', 'معالجة', 'المدفوعات', 'الدولية', 'لمزود', 'تكنولوجيا', 'التعليم', 'الجديد', 'لدينا', '.', 'هل', 'يمكنك', 'التحقق', 'من', 'المشكلة', 'مع', 'BIC', 'LERQNZJBXXX', 'و', 'IBAN', 'NO2484520800325', '؟']
<class 'list'> 28
['O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-BIC', 'O', 'O', 'B-IBAN', 'O']
<class 'list'> 28


In [11]:
# Convert tokens and tags columns to lists

train['tokens'] = train["tokens"].apply(ast.literal_eval)
train['tags'] = train['tags'].apply(ast.literal_eval)

In [12]:
train.head()

,source,tokens,tags,clean_source
0,مرحبًا <!FIRSTNAME_1!> Johan </!FIRSTNAME_1!> ...,"[مرحبا, Johan, Walker, ،, إشعار, الامتثال, :, ...","[O, B-FIRSTNAME, B-LASTNAME, O, O, O, O, O, O,...",مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...
1,نحن نقوم بمراجعة القضية التي رفعها <!FIRSTNAME...,"[نحن, نقوم, بمراجعة, القضية, التي, رفعها, كادي...","[O, O, O, O, O, O, B-FIRSTNAME, B-LASTNAME, O,...",نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...
2,بسبب اتفاقية السرية، لا يمكن للمريضة <!SEX_1!>...,"[بسبب, اتفاقية, السرية, ،, لا, يمكن, للمريضة, ...","[O, O, O, O, O, O, O, B-SEX, O, O, O, B-PREFIX...",بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في <!DATE...,"[عزيزي, ولي, الأمر, ،, يلزم, الحصول, على, إذن,...","[O, O, O, O, O, O, O, O, O, B-DATE, I-DATE, I-...",عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...
4,تنظيم اجتماع حول <!AGE_1!> 7 سنوات </!AGE_1!> ...,"[تنظيم, اجتماع, حول, 7, سنوات, (, Genderqueer,...","[O, O, O, B-AGE, I-AGE, O, B-GENDER, O, O, O, ...",تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...


In [28]:
length_comparison_result = train.apply(lambda row: len(train.tokens) == len(train.tags), axis=1)
print(f"\nNumber of rows with mismatching lengths: {sum(~length_comparison_result)}")


Number of rows with mismatching lengths: 0


In [29]:
tags_to_mask =  [
'EMAIL',
'IBAN',
'PERSONNAME',
'IPV6',
'MASKEDNUMBER',
'CREDITCARDNUMBER',
'LITECOINADDRESS',
'BIC',
'PHONENUMBER',
'ORDINALDIRECTION',
'VEHICLEVRM',
'CREDITCARDINUMBER',
'USERNAME',
'IPV4',
'PASSWORD',
'MAC',
'BITCOINADDRESS',
'SECONDARYADDRESS',
'CREDITCARDCVV',
'PHONEIMEI',
'ACCOUNTNAME',
'BUILDINGNUMBER',
'VEHICLEVIN',
'STREET',
'URL',
'IP',
'ZIPCODE',
'ACCOUNTNUMBER',
'FIRSTNAME',
'LASTNAME',
'MIDDLENAME',
'NEARBYGPSCOORDINATE',
'ETHEREUMADDRESS',
'PIN',
'SSN',
'COMPANYNAME',
'HEIGHT',
'CITY']

In [30]:
def masker(row, tags_to_mask):
    """
    Constructs a masked sentence by iterating through tokens and replacing PII based on tags, attempting to preserve original formatting.

    Args:
    row: A pandas DataFrame row with 'tokens', 'tags', and 'clean_source' columns.
    tags_to_mask: List of tag entity types to mask.

    Returns:
    The constructed sentence with masked tokens for the given row, attempting to preserve spacing.
    """
    tokens = row['tokens']
    tags = row['tags']
    clean_source = row['clean_source']
    masked_sentence = ""
    current_position = 0

    for i in range(len(tokens)):
        token = tokens[i]
        tag = tags[i]

        # Find the position of the current token in the original string starting from the last processed position
        start_index = clean_source.find(token, current_position)

        if start_index != -1:
            # Append the text from the current position up to the start of the token
            masked_sentence += clean_source[current_position:start_index]

            if tag[2:] in tags_to_mask:
                masked_sentence += '[MASK]'
            else:
                masked_sentence += token

            # Update the current position to the end of the current token
            current_position = start_index + len(token)
        else:
            # If token not found at or after current_position, it indicates a discrepancy
            print(f"Warning: Token '{token}' not found in clean_source at or after position {current_position}")

    # Append any remaining text after the last token
    masked_sentence += clean_source[current_position:]

    return masked_sentence

In [31]:
train['masked_source'] = train.apply(lambda row: masker(row, tags_to_mask), axis=1)

In [32]:
train[['clean_source','masked_source']]

,clean_source,masked_source
0,مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...,مرحبا [MASK] [MASK]، إشعار الامتثال: تتطلب الت...
1,نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...,نحن نقوم بمراجعة القضية التي رفعها [MASK] [MAS...
2,بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...,بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...,عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...
4,تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...,تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...
...,...,...
209256,عزيزتي الأنثى، حدث التعليم الفني في الشقة. تم ...,عزيزتي الأنثى، حدث التعليم الفني في [MASK][MAS...
209257,هل يمكننا ترتيب موعد مع والدي نيكو؟ لقد طلبت إ...,هل يمكننا ترتيب موعد مع والدي [MASK]؟ لقد طلبت...
209258,يرجى تأكيد استلام الزي الجديد عن طريق البريد ا...,يرجى تأكيد استلام الزي الجديد عن طريق البريد ا...
209259,أظهر الشخص ذو العيون ذات اللون البني وطوله 3' ...,أظهر الشخص ذو العيون ذات اللون البني وطوله [MA...


No charts were generated by quickchart


In [40]:
train['tokens'][20]

['ترقيات',
 'المختبر',
 'في',
 'مدرستنا',
 'في',
 'إيست',
 'بيرسي',
 '.',
 'هل',
 'يمكننا',
 'الحصول',
 'على',
 'عنوان',
 'IP',
 '141',
 '.',
 '247',
 '.',
 '15',
 '.',
 '141',
 'وعنوان',
 'MAC',
 '8f',
 ':',
 '2f',
 ':',
 'd6',
 ':',
 '6f',
 ':',
 'f3',
 ':',
 '31',
 'للتكوين',
 '؟']

In [41]:
train['tags'][20]

['O',
 'O',
 'O',
 'O',
 'O',
 'B-CITY',
 'I-CITY',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-IP',
 'I-IP',
 'I-IP',
 'I-IP',
 'I-IP',
 'I-IP',
 'I-IP',
 'O',
 'O',
 'B-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'I-MAC',
 'O',
 'O']

In [42]:
train.clean_source[20]

'ترقيات المختبر في مدرستنا في إيست بيرسي. هل يمكننا الحصول على عنوان IP 141.247.15.141 وعنوان MAC 8f:2f:d6:6f:f3:31 للتكوين؟'

In [37]:
masker(train.iloc[7], tags_to_mask)

'عزيزي [MASK]، لقد أعجبت الإدارة بأفكارك. نود أن نناقش بشكل أكبر استراتيجية العلاقات العامة في [MASK] [MASK] [MASK] [MASK]. لا تتردد في الاتصال بنا على [MASK][MASK][MASK][MASK][MASK].'

---

---



---



---



##### Use only the needed tags

In [ ]:
train['masked_tags'] = train['tags'].apply(lambda x: mask_tags(x, tags_to_mask))

In [ ]:
train['masked_tags'][5:11]

,masked_tags
5,"[O, [MASKED], O, O, O, O, O, O, O, O, O, O, [M..."
6,"[O, O, O, O, O, O, O, O, O, O, O, O, [MASKED],..."
7,"[O, [MASKED], O, O, O, O, O, O, O, O, O, O, O,..."
8,"[O, O, O, O, O, O, O, O, [MASKED], [MASKED], O..."
9,"[O, O, O, O, O, O, O, O, O, [MASKED], O, O, O,..."
10,"[O, O, O, O, O, O, O, [MASKED], [MASKED], [MAS..."


In [ ]:
print(train['masked_tags'][0])
print(train['tokens'][0])
print(train['tags'][0])

['O', '[MASKED]', '[MASKED]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[MASKED]', '[MASKED]', 'O', 'O', 'O', 'O', 'O']
['مرحبا', 'Johan', 'Walker', '،', 'إشعار', 'الامتثال', ':', 'تتطلب', 'التغييرات', 'الأخيرة', 'في', 'التشريعات', 'في', 'سانت', 'غالن', 'منك', 'تحديث', 'Suite', '559', 'الخاص', 'بك', 'في', 'سجلاتنا', '.']
['O', 'B-FIRSTNAME', 'B-LASTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-STATE', 'I-STATE', 'O', 'O', 'B-SECONDARYADDRESS', 'I-SECONDARYADDRESS', 'O', 'O', 'O', 'O', 'O']


In [ ]:
print(train['masked_tags'][3])
print(train['tokens'][3])
print(train['tags'][3])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[MASKED]', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['عزيزي', 'ولي', 'الأمر', '،', 'يلزم', 'الحصول', 'على', 'إذن', 'في', '2018', '-', '04', '-', '17T03', ':', '32', ':', '46', '.', '898Z', 'للرحلة', 'المدرسية', 'لـ', 'Maximillian', '.', 'يرجى', 'الرجوع', 'إلى', 'النموذج', 'المرفق', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
length_comparison_result = train.apply(lambda row: len(train.tokens) == len(train.masked_tags), axis=1)
print(f"\nNumber of rows with mismatching lengths: {sum(~length_comparison_result)}")


Number of rows with mismatching lengths: 0


##### Creating new masked sentences

In [ ]:
import re

In [ ]:
def create_masked_sentence(row):
    """Creates a masked sentence from tokens and masked tags.

    Args:
    row: A pandas DataFrame row with 'tokens' and 'masked_tags' columns.

    Returns:
    A string representing the masked sentence.
    """
    masked_sentence_tokens = []
    for token, tag in zip(row['tokens'], row['masked_tags']):
        if tag == '[MASKED]':
            masked_sentence_tokens.append('[masked]')
        else:
            masked_sentence_tokens.append(token)

    return ' '.join(masked_sentence_tokens)

In [ ]:
# Apply the function to each row to create the new masked sentence column
train['masked_source'] = train.apply(create_masked_sentence, axis=1)

In [ ]:
print(train['masked_source'][3])
print(train['clean_source'][3])
print(train['masked_tags'][3])
print(train['tokens'][3])
print(train['tags'][3])

عزيزي ولي الأمر ، يلزم الحصول على إذن في 2018 - 04 - 17T03 : 32 : 46 . 898Z للرحلة المدرسية لـ [masked] . يرجى الرجوع إلى النموذج المرفق .
عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-04-17T03:32:46.898Z للرحلة المدرسية لـ Maximillian. يرجى الرجوع إلى النموذج المرفق.
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[MASKED]', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['عزيزي', 'ولي', 'الأمر', '،', 'يلزم', 'الحصول', 'على', 'إذن', 'في', '2018', '-', '04', '-', '17T03', ':', '32', ':', '46', '.', '898Z', 'للرحلة', 'المدرسية', 'لـ', 'Maximillian', '.', 'يرجى', 'الرجوع', 'إلى', 'النموذج', 'المرفق', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'B-FIRSTNAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
train.head()

,source,tokens,tags,clean_source,masked_tags,masked_source
0,مرحبًا <!FIRSTNAME_1!> Johan </!FIRSTNAME_1!> ...,"[مرحبا, Johan, Walker, ،, إشعار, الامتثال, :, ...","[O, B-FIRSTNAME, B-LASTNAME, O, O, O, O, O, O,...",مرحبا Johan Walker، إشعار الامتثال: تتطلب التغ...,"[O, [MASKED], [MASKED], O, O, O, O, O, O, O, O...",مرحبا [masked] [masked] ، إشعار الامتثال : تتط...
1,نحن نقوم بمراجعة القضية التي رفعها <!FIRSTNAME...,"[نحن, نقوم, بمراجعة, القضية, التي, رفعها, كادي...","[O, O, O, O, O, O, B-FIRSTNAME, B-LASTNAME, O,...",نحن نقوم بمراجعة القضية التي رفعها كادي شوبي ب...,"[O, O, O, O, O, O, [MASKED], [MASKED], O, O, O...",نحن نقوم بمراجعة القضية التي رفعها [masked] [m...
2,بسبب اتفاقية السرية، لا يمكن للمريضة <!SEX_1!>...,"[بسبب, اتفاقية, السرية, ،, لا, يمكن, للمريضة, ...","[O, O, O, O, O, O, O, B-SEX, O, O, O, B-PREFIX...",بسبب اتفاقية السرية، لا يمكن للمريضة أنثى الدر...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]",بسبب اتفاقية السرية ، لا يمكن للمريضة أنثى الد...
3,عزيزي ولي الأمر، يلزم الحصول على إذن في <!DATE...,"[عزيزي, ولي, الأمر, ،, يلزم, الحصول, على, إذن,...","[O, O, O, O, O, O, O, O, O, B-DATE, I-DATE, I-...",عزيزي ولي الأمر، يلزم الحصول على إذن في 2018-0...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",عزيزي ولي الأمر ، يلزم الحصول على إذن في 2018 ...
4,تنظيم اجتماع حول <!AGE_1!> 7 سنوات </!AGE_1!> ...,"[تنظيم, اجتماع, حول, 7, سنوات, (, Genderqueer,...","[O, O, O, B-AGE, I-AGE, O, B-GENDER, O, O, O, ...",تنظيم اجتماع حول 7 سنوات (Genderqueer) وسلامة ...,"[O, O, O, O, O, O, O, O, O, O, O, O, [MASKED],...",تنظيم اجتماع حول 7 سنوات ( Genderqueer ) وسلام...


In [ ]:
# Save the train data after the updates
train.to_csv('/content/drive/MyDrive/Colab Notebooks/ArabicPIIRedaction/data/masked_train_data.csv', index=False)

# Loading Model

### SILMA-2B-4bit

In [ ]:
# !pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
model_id = "silma-ai/SILMA-Kashif-2B-Instruct-v1.0-bnb-4bit" # Quantized version
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = quantization_config,
)

In [ ]:
messages = [
    {"role": "user", "content": "كم عدد أيام الأسبوع؟"},
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

input_ids

In [ ]:
outputs = model.generate(**input_ids, max_new_tokens=256)

In [ ]:
tokenizer.decode(outputs[0]).split("<start_of_turn>")[-1]

### SILMA-2B

In [ ]:
import torch
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model="silma-ai/SILMA-Kashif-2B-Instruct-v1.0",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda"
)

In [ ]:
messages = [
    {"role": "user",
     "content": """ Mask the PII as [masked] in the given source sentence

     source: I sent my email address M.helmy@nu.edu.eg to my supervisor dr. gahda khoriba in Nile University.

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
response

In [ ]:
messages = [
    {"role": "user",
     "content": f""" Mask the PII as [masked] in the given source sentence

     source: {df["source"][0]}

     target:
     """
     }
]

outputs = pipe(messages, max_new_tokens=600)
response = outputs[0]["generated_text"][-1]["content"].strip()

In [ ]:
print(df["source"][0])
print(response)